In [1]:
import mlflow
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing
import catboost as cb
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns

In [2]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams.update({'font.size': 15})

In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("income")

2023/11/26 11:21:19 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/11/26 11:21:19 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='file:///c:/Users/Lenovo/Downloads/mlruns/1', creation_time=1700994079899, experiment_id='1', last_update_time=1700994079899, lifecycle_stage='active', name='income', tags={}>

## Load Data

In [4]:
dset = fetch_california_housing()
data = dset['data']
y = dset['target']
LABEL = dset['target_names'][0]

NUMERIC_FEATURES = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Longitude', 'Latitude']
FEATURES = NUMERIC_FEATURES

data = pd.DataFrame(data, columns=dset['feature_names'])
data[LABEL] = y

data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [5]:
train_data, test_data = train_test_split(data, test_size=0.2)
print(f"Train dataset shape: {train_data.shape}")
print(f"Test dataset shape: {test_data.shape}")

Train dataset shape: (16512, 9)
Test dataset shape: (4128, 9)


## Data Processing

In [6]:
X_train, X_val = train_test_split(train_data, test_size=0.2)

sc = StandardScaler()
X_train.loc[:, NUMERIC_FEATURES] = sc.fit_transform(X_train[NUMERIC_FEATURES])
X_val.loc[:, NUMERIC_FEATURES] = sc.transform(X_val[NUMERIC_FEATURES])
test_data.loc[:, NUMERIC_FEATURES] = sc.transform(test_data[NUMERIC_FEATURES])

## Baseline

In [7]:
mlflow.sklearn.autolog(disable=True)

with mlflow.start_run(run_name='rf_baseline'):
    params = {
        "n_estimators": 100,
        "max_depth": 20
    }

    mlflow.set_tag("model_name", "RF")
    mlflow.log_params(params)

    rf = RandomForestRegressor(n_estimators=100, max_depth=20)
    rf.fit(X_train[FEATURES], X_train[LABEL])

    rf_preds = rf.predict(test_data[FEATURES])
    rf_rms = mean_squared_error(test_data[LABEL], rf_preds, squared=False)

    mlflow.log_metric("test_rmse", rf_rms)
    mlflow.sklearn.log_model(rf, "sk_models")

c:\Users\Lenovo\anaconda3\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


## CatBoost

In [8]:
catb_train_dataset = cb.Pool(X_train[FEATURES], X_train[LABEL]) 
catb_val_dataset = cb.Pool(X_val[FEATURES], X_val[LABEL]) 
catb_test_dataset = cb.Pool(test_data[FEATURES], test_data[LABEL])

In [9]:
with mlflow.start_run(run_name="catboost"):
    mlflow.set_tag("model_name", "CatBoost")
    catb = cb.CatBoostRegressor()
    catb.fit(catb_train_dataset, eval_set=catb_val_dataset, early_stopping_rounds=50)
    catb_preds = catb.predict(catb_test_dataset)
    catb_rms = mean_squared_error(test_data[LABEL], catb_preds, squared=False)

    mlflow.log_metric("test_rmse", catb_rms)
    mlflow.catboost.log_model(catb, "cb_models")

Learning rate set to 0.076361
0:	learn: 1.1093814	test: 1.0984648	best: 1.0984648 (0)	total: 158ms	remaining: 2m 38s
1:	learn: 1.0659563	test: 1.0563419	best: 1.0563419 (1)	total: 176ms	remaining: 1m 27s
2:	learn: 1.0246741	test: 1.0157631	best: 1.0157631 (2)	total: 188ms	remaining: 1m 2s
3:	learn: 0.9885720	test: 0.9811210	best: 0.9811210 (3)	total: 200ms	remaining: 49.7s
4:	learn: 0.9550396	test: 0.9497903	best: 0.9497903 (4)	total: 211ms	remaining: 42.1s
5:	learn: 0.9248258	test: 0.9203138	best: 0.9203138 (5)	total: 216ms	remaining: 35.8s
6:	learn: 0.8983365	test: 0.8953379	best: 0.8953379 (6)	total: 222ms	remaining: 31.5s
7:	learn: 0.8724961	test: 0.8703530	best: 0.8703530 (7)	total: 227ms	remaining: 28.1s
8:	learn: 0.8504201	test: 0.8488020	best: 0.8488020 (8)	total: 231ms	remaining: 25.5s
9:	learn: 0.8288904	test: 0.8274709	best: 0.8274709 (9)	total: 234ms	remaining: 23.2s
10:	learn: 0.8093905	test: 0.8093999	best: 0.8093999 (10)	total: 238ms	remaining: 21.4s
11:	learn: 0.7888623